In [1]:
import pandas as pd
import unicodedata
from pathlib import Path

DATA_PATH = Path('dados_tratados_completo.csv')
OUTPUT_DIR = Path('.')
if not DATA_PATH.exists():
    raise FileNotFoundError('dados_tratados.csv nao encontrado na raiz do projeto')

def read_csv_with_fallback(path):
    for enc in ('utf-8', 'cp1252', 'latin1'):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, low_memory=False)

def write_csv(df, name, index=False, index_label=None):
    df.to_csv(OUTPUT_DIR / name, index=index, index_label=index_label, encoding='utf-8')

df = read_csv_with_fallback(DATA_PATH)
df.head()


,uf,mes_referencia,idade,sexo,escolaridade,teve_febre,teve_tosse,teve_dor_garganta,teve_dificuldade_respirar,teve_dor_peito,perda_olfato_paladar,procurou_saude,resultado_exame,medida_isolamento,trabalhou_semana_passada,afastado_trabalho,tipo_trabalho,faixa_rendimento,trabalho_remoto,auxilio_emergencial
0,Rondônia,7,35,Homem,Médio completo,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Sim,Não aplicável,Outro técnico ou profissional de nível médio,801 - 1.600,Não aplicável,Sim
1,Rondônia,7,29,Mulher,Superior completo,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou em casa e só saiu em caso de necessidade...,Não,Sim,não aplicável,Não aplicável,Não aplicável,Sim
2,Rondônia,7,13,Homem,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
3,Rondônia,7,10,Homem,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
4,Rondônia,7,57,Mulher,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou em casa e só saiu em caso de necessidade...,Não,Não,não aplicável,Não aplicável,Não aplicável,Não


In [2]:
# padroniza textos
obj_cols = df.select_dtypes(include='object').columns
df[obj_cols] = df[obj_cols].apply(lambda s: s.str.strip())

df['idade'] = pd.to_numeric(df['idade'], errors='coerce')

preferred_sintomas = [
    'teve_febre',
    'teve_tosse',
    'teve_dor_garganta',
    'teve_dificuldade_respirar',
    'teve_dor_peito',
    'perda_olfato_paladar',
]

extra_sintomas = sorted(
    c
    for c in df.columns
    if (c.startswith('teve_') or c == 'perda_olfato_paladar') and c not in preferred_sintomas
)
sintoma_cols = [c for c in preferred_sintomas if c in df.columns] + extra_sintomas

if not sintoma_cols:
    raise ValueError('Nenhuma coluna de sintoma encontrada no dataset.')

bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, 120]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['faixa_etaria'] = pd.cut(df['idade'], bins=bins, labels=labels, include_lowest=True)

df['tem_sintoma_chave'] = df[sintoma_cols].eq('Sim').any(axis=1)

def to_ascii(text):
    return (
        unicodedata.normalize('NFKD', text)
        .encode('ascii', 'ignore')
        .decode('ascii')
        .lower()
        .replace(' ', '_')
    )


In [3]:
sym_long_freq = df.melt(
    value_vars=sintoma_cols,
    var_name='sintoma',
    value_name='resposta',
)
sym_long_freq['resposta_norm'] = sym_long_freq['resposta'].map(lambda s: to_ascii(str(s)))

freq_counts = (
    sym_long_freq.groupby(['resposta_norm', 'sintoma'])
    .size()
    .reset_index(name='contagem')
)
freq_counts['pct'] = (
    freq_counts.groupby('sintoma')['contagem']
    .transform(lambda s: (s / s.sum()) * 100)
    .round(6)
)

frequencia_sintomas = freq_counts.pivot_table(
    index='resposta_norm',
    columns='sintoma',
    values='pct',
    fill_value=0,
)

resposta_order = ['nao', 'sim', 'ignorado', 'nao_sabe']
resposta_labels = {
    'nao': 'Nao',
    'sim': 'Sim',
    'ignorado': 'Ignorado',
    'nao_sabe': 'Nao_sabe',
}
sintoma_labels = {
    'teve_dificuldade_respirar': 'Dificuldade de respirar',
    'teve_febre': 'Teve Febre',
    'teve_tosse': 'Teve Tosse',
    'teve_dor_garganta': 'Teve dor de garganta',
    'teve_dor_peito': 'Teve dor no peito',
    'perda_olfato_paladar': 'Perda de olfato e paladar',
}

def default_sintoma_label(col):
    label = col.replace('_', ' ')
    if label.startswith('teve '):
        label = 'Teve ' + label[len('teve '):]
    elif label.startswith('perda '):
        label = 'Perda ' + label[len('perda '):]
    return label.title()

sintoma_labels_full = {
    col: sintoma_labels.get(col, default_sintoma_label(col))
    for col in sintoma_cols
}

frequencia_sintomas = frequencia_sintomas.reindex(columns=sintoma_cols)
frequencia_sintomas = frequencia_sintomas.reindex(resposta_order)
frequencia_sintomas = frequencia_sintomas.rename(index=resposta_labels)
frequencia_sintomas = frequencia_sintomas.rename(columns=sintoma_labels_full)

write_csv(frequencia_sintomas, 'frequencia_sintomas.csv', index=True, index_label='resposta')

frequencia_sintomas


sintoma,Teve Febre,Teve Tosse,Teve dor de garganta,Dificuldade de respirar,Teve dor no peito,Perda de olfato e paladar
resposta_norm,,,,,,
Nao,98.318543,97.614993,97.992287,98.858706,99.007067,98.712504
Sim,1.191381,1.889922,1.505461,0.639905,0.484204,0.656658
Ignorado,0.417450,0.421595,0.421077,0.425222,0.425308,0.545690
Nao_sabe,0.072626,0.073490,0.081176,0.076167,0.083421,0.085148


In [4]:
sym_long = df.melt(
    id_vars=['mes_referencia'],
    value_vars=sintoma_cols,
    var_name='sintoma',
    value_name='resposta',
)
sym_long['resposta_norm'] = sym_long['resposta'].map(lambda s: to_ascii(str(s)))
sym_long['resposta_norm'] = sym_long['resposta_norm'].replace({'nao_sabe': 'ignorado'})

sym_counts = (
    sym_long.groupby(['mes_referencia', 'sintoma', 'resposta_norm'])
    .size()
    .reset_index(name='contagem')
)
sym_counts['pct'] = (
    sym_counts.groupby(['mes_referencia', 'sintoma'])['contagem']
    .transform(lambda s: (s / s.sum()).round(4))
)

gold_sintomas_mes = sym_counts.pivot_table(
    index=['mes_referencia', 'sintoma'],
    columns='resposta_norm',
    values=['contagem', 'pct'],
    fill_value=0,
)
gold_sintomas_mes.columns = [
    f"{metric}_{resp}" for metric, resp in gold_sintomas_mes.columns
]
gold_sintomas_mes = gold_sintomas_mes.reset_index()

for resp in ['ignorado', 'nao', 'sim']:
    for metric in ['contagem', 'pct']:
        col = f'{metric}_{resp}'
        if col not in gold_sintomas_mes.columns:
            gold_sintomas_mes[col] = 0

gold_sintomas_mes = gold_sintomas_mes[[
    'mes_referencia',
    'sintoma',
    'contagem_ignorado',
    'contagem_nao',
    'contagem_sim',
    'pct_ignorado',
    'pct_nao',
    'pct_sim',
]]

gold_sintomas_mes.head()


,mes_referencia,sintoma,contagem_ignorado,contagem_nao,contagem_sim,pct_ignorado,pct_nao,pct_sim
0,7,perda_olfato_paladar,2660.0,378112.0,3394.0,0.0069,0.9842,0.0088
1,7,teve_dificuldade_respirar,2055.0,379069.0,3042.0,0.0053,0.9867,0.0079
2,7,teve_dor_garganta,2062.0,375347.0,6757.0,0.0054,0.9770,0.0176
3,7,teve_dor_peito,2087.0,379657.0,2422.0,0.0054,0.9883,0.0063
4,7,teve_febre,2008.0,376326.0,5832.0,0.0052,0.9796,0.0152


In [5]:
gold_sintomas_perfil = (
    df.groupby(['mes_referencia', 'faixa_etaria', 'sexo', 'uf'])
    .agg(total=('tem_sintoma_chave', 'size'), com_sintoma=('tem_sintoma_chave', 'sum'))
    .reset_index()
)
gold_sintomas_perfil['pct_com_sintoma_chave'] = (
    (gold_sintomas_perfil['com_sintoma'] / gold_sintomas_perfil['total']).round(4)
)
gold_sintomas_perfil.head()


C:\Users\00157NLUC-BrenoR\AppData\Local\Temp\ipykernel_12776\1550526282.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['mes_referencia', 'faixa_etaria', 'sexo', 'uf'])


,mes_referencia,faixa_etaria,sexo,uf,total,com_sintoma,pct_com_sintoma_chave
0,7,0-9,Homem,Acre,497,13,0.0262
1,7,0-9,Homem,Alagoas,855,26,0.0304
2,7,0-9,Homem,Amapá,220,6,0.0273
3,7,0-9,Homem,Amazonas,720,18,0.0250
4,7,0-9,Homem,Bahia,1040,28,0.0269


In [6]:
economic_cols = [
    'trabalhou_semana_passada',
    'afastado_trabalho',
    'tipo_trabalho',
    'faixa_rendimento',
    'trabalho_remoto',
    'auxilio_emergencial',
]
economic_cols = [c for c in economic_cols if c in df.columns]

econ_long = df.melt(
    id_vars=['mes_referencia'],
    value_vars=economic_cols,
    var_name='indicador_economico',
    value_name='categoria',
)

gold_economia_mes = (
    econ_long.groupby(['mes_referencia', 'indicador_economico', 'categoria'])
    .size()
    .reset_index(name='contagem')
)
gold_economia_mes['pct'] = (
    gold_economia_mes.groupby(['mes_referencia', 'indicador_economico'])['contagem']
    .transform(lambda s: (s / s.sum()).round(4))
)
gold_economia_mes.head()


,mes_referencia,indicador_economico,categoria,contagem,pct
0,7,afastado_trabalho,Não,162634,0.4233
1,7,afastado_trabalho,Não aplicável,197526,0.5142
2,7,afastado_trabalho,Sim,24006,0.0625
3,7,auxilio_emergencial,Não,184673,0.4807
4,7,auxilio_emergencial,Sim,199493,0.5193


In [7]:
behavior_cols = ['medida_isolamento', 'procurou_saude']
behavior_cols = [c for c in behavior_cols if c in df.columns]

behavior_long = df.melt(
    id_vars=['mes_referencia'],
    value_vars=behavior_cols,
    var_name='indicador_comportamento',
    value_name='categoria',
)

gold_comportamento_mes = (
    behavior_long.groupby(['mes_referencia', 'indicador_comportamento', 'categoria'])
    .size()
    .reset_index(name='contagem')
)
gold_comportamento_mes['pct'] = (
    gold_comportamento_mes.groupby(['mes_referencia', 'indicador_comportamento'])['contagem']
    .transform(lambda s: (s / s.sum()).round(4))
)



gold_sintomas_frequentes_mes = gold_sintomas_mes[[
    'mes_referencia',
    'sintoma',
    'pct_sim',
]].copy()

gold_sintomas_frequentes_mes['rank_pct_sim'] = (
    gold_sintomas_frequentes_mes.groupby('mes_referencia')['pct_sim']
    .rank(method='dense', ascending=False)
    .astype(int)
)
write_csv(gold_sintomas_mes, 'gold_sintomas_mes.csv')
write_csv(gold_sintomas_perfil, 'gold_sintomas_perfil.csv')
write_csv(gold_economia_mes, 'gold_economia_mes.csv')
write_csv(gold_comportamento_mes, 'gold_comportamento_mes.csv')
write_csv(gold_sintomas_frequentes_mes, 'gold_sintomas_frequentes_mes.csv')

gold_comportamento_mes.head()


,mes_referencia,indicador_comportamento,categoria,contagem,pct
0,7,medida_isolamento,Ficou em casa e só saiu em caso de necessidade...,172452,0.4489
1,7,medida_isolamento,Ficou rigorosamente em casa,89634,0.2333
2,7,medida_isolamento,Ignorado,2292,0.0060
3,7,medida_isolamento,"Não fez restrição, levou vida normal como ante...",7195,0.0187
4,7,medida_isolamento,"Reduziu o contato com as pessoas, mas continuo...",112593,0.2931
